In [1]:
# Importação das bibliotecas
import pandas as pd
from sklearn import linear_model as lm
from sklearn import metrics as mt
import numpy as np

In [2]:
def calculate_metrics(y, yhat):
    '''
        Recebe as targets original e prevista, calcula as métricas e retorna um dicionário
    '''
    mse = round( mt.mean_squared_error( y, yhat ), 3)
    metrics = {'R2': round( mt.r2_score( y, yhat ), 3),
           'MSE': mse,
           'RMSE': round( np.sqrt( mse ), 3),
           'MAE': round( mt.mean_absolute_error( y, yhat ), 3),
           'MAPE': round( mt.mean_absolute_percentage_error( y, yhat ), 3)}   

    return metrics

In [3]:
# Features selecionadas
features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']

In [4]:
# Carregamento dos dados de treino
X_train = pd.read_csv( 'X_training.csv' )
y_train = pd.read_csv( 'y_training.csv' )

# Preparação dos dados de treino
X_train = X_train.loc[:, features]
y_train = y_train.values.ravel()

In [5]:
# Carregamento dos dados de validação:
X_val = pd.read_csv( 'X_validation.csv' )
y_val = pd.read_csv( 'y_val.csv' )

# Preparação dos dados de validação
X_val = X_val.loc[:, features]
y_val = y_val.values.ravel()

In [6]:
# Carregamento dos dados de teste
X_test = pd.read_csv( 'X_test.csv' )
y_test = pd.read_csv( 'y_test.csv' )

# Preparação dos dados de teste
X_test = X_test.loc[:, features]
y_test = y_test.values.ravel()

In [7]:
# Treinamento do modelo com os parâmetros padrão do algoritmo
model = lm.LinearRegression()
model.fit( X_train, y_train )

LinearRegression()

In [8]:
# Especialmente neste caso da Regressão Linear não vamos variar os parâmetros.
# Vamos utilizar somente os valores de parâmetros padrão para via de comparação com as demais regressões lineares
# (Ridge, Lasso e Elastic Net)

df_scores_final = pd.DataFrame(columns=['dataset','R2', 'MSE', 'RMSE', 'MAE', 'MAPE'])

In [9]:
# Cálculo das métricas para os dados de treinamento com os parametros padrão, a objetivo de comparação

yhat_train = model.predict( X_train)
res = calculate_metrics(y_train, yhat_train)

df_scores_final = df_scores_final._append({'dataset': 'Treinamento', 'R2': res['R2'], 'MSE': res['MSE'], 'RMSE': res['RMSE'],
                                               'MAE': res['MAE'], 'MAPE': res['MAPE'] }, ignore_index=True) 


C:\Users\gabri\AppData\Local\Temp\ipykernel_13772\2446607437.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_scores_final = df_scores_final._append({'dataset': 'Treinamento', 'R2': res['R2'], 'MSE': res['MSE'], 'RMSE': res['RMSE'],


In [10]:
# Cálculo das métricas para os dados de validação com os parametros padrão, a objetivo de comparação
yhat_val = model.predict( X_val)
res = calculate_metrics(y_val, yhat_val)

df_scores_final = df_scores_final._append({'dataset': 'Validacao', 'R2': res['R2'], 'MSE': res['MSE'], 'RMSE': res['RMSE'],
                                               'MAE': res['MAE'], 'MAPE': res['MAPE'] }, ignore_index=True)

In [11]:
# Concatenar os dados de treino e validação para uso da técnica de validação holdout com os dados de teste
X_trainval = pd.concat([X_train, X_val], ignore_index=True)
y_trainval = np.concatenate((y_train, y_val), axis=None)

In [12]:
# Performance de teste:
model = lm.ElasticNet()
model.fit( X_trainval, y_trainval )

yhat_test = model.predict( X_test)
res = calculate_metrics(y_test, yhat_test)

df_scores_final = df_scores_final._append({'dataset': 'Teste', 'R2': res['R2'], 'MSE': res['MSE'], 'RMSE': res['RMSE'],
                                               'MAE': res['MAE'], 'MAPE': res['MAPE'] }, ignore_index=True)

In [13]:
df_scores_final

,dataset,R2,MSE,RMSE,MAE,MAPE
0,Treinamento,0.046,455.996,21.354,16.998,8.653
1,Validacao,0.040,458.447,21.411,17.040,8.683
2,Teste,0.008,483.056,21.979,17.472,8.746
